# Exploring Airports in Australia

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config_key import geoapify_key

## DataFrame Preparation

In [2]:
# Import cities file as DataFrame
cities_pd = pd.read_csv("../Resources/cities.csv")
cities_pd

,City
0,Sydney
1,Melbourne
2,Perth
3,Cairns
4,Adelaide
5,Gold Coast
6,Townsville
7,Williamtown


In [3]:
# Add columns the airports data we will fetch using the Geoapify API
# Note that we used "" to specify initial entry.
cities_pd['Lat'] = ""
cities_pd["Lon"] = ""
cities_pd["Airport Name"] = ""
cities_pd["IATA Name"] = ""
cities_pd["Airport Address"] = ""
cities_pd["Distance"] = ""
cities_pd["Website"] = ""
cities_pd

,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,,,,,,,
1,Melbourne,,,,,,,
2,Perth,,,,,,,
3,Cairns,,,,,,,
4,Adelaide,,,,,,,
5,Gold Coast,,,,,,,
6,Townsville,,,,,,,
7,Williamtown,,,,,,,


## Look for Geographical Coordinates for Each City

In [4]:
# Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [5]:
from pprint import pprint
response = requests.get(base_url, params=params)
response.url

'https://api.geoapify.com/v1/geocode/search?apiKey=81c15e438afc4984a110b434a5c7e2d3&format=json'

In [6]:
# Print a message to follow up the airport search
print("Starting airport search")

# Loop through the cities_pd DataFrame and search coordinates for each city
for index, row in cities_pd.iterrows():

    # Get the city's name & add ", Australia" to the string so geoapify finds the correct city
    city = row["City"] + ", Australia"

    # Add the current city to the parameters
    params["text"] = f"{city}"

    # Make the API request
    response = requests.get(base_url, params=params)
    
    # Convert response to JSON
    response = response.json()

    # Extract latitude and longitude
    # Extract latitude and longitude
    lat = response["results"][0]["lat"]
    lon = response["results"][0]["lon"]

    # update dataframe values 
    cities_pd.loc[index,"Lat"] = lat
    cities_pd.loc[index,"Lon"] = lon
    
    # Log the search results
    print(f"Coordinates for {city} fetched...")

# Display sample data to confirm that the coordinates appear
cities_pd

Starting airport search
Coordinates for Sydney, Australia fetched...
Coordinates for Melbourne, Australia fetched...
Coordinates for Perth, Australia fetched...
Coordinates for Cairns, Australia fetched...
Coordinates for Adelaide, Australia fetched...
Coordinates for Gold Coast, Australia fetched...
Coordinates for Townsville, Australia fetched...
Coordinates for Williamtown, Australia fetched...


,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,,,,,
1,Melbourne,-37.814245,144.963173,,,,,
2,Perth,-31.955893,115.860585,,,,,
3,Cairns,-16.920666,145.772185,,,,,
4,Adelaide,-34.928181,138.599931,,,,,
5,Gold Coast,-28.002373,153.414599,,,,,
6,Townsville,-19.256939,146.823954,,,,,
7,Williamtown,-32.815,151.842778,,,,,


## Retrieve Airports' Information

In [9]:
# Set parameters to search for airport's details
radius = 50000
params = {
    "categories":"airport",
    "limit":20,
    "apiKey": geoapify_key
    
}

# Print a message to follow up the airport search
print("Starting airport details search")

# Iterate through the types_df DataFrame
for index, row in cities_pd.iterrows():

    # Get the city's name
    city = row["City"]

    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lon"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    response = response.json()

    # Grab the first airport from the results and store the details in the DataFrame
    try:
        cities_pd.loc[index, "Airport Name"] = response["features"][1]["properties"]["name"]
        cities_pd.loc[index, "IATA Name"] = response["features"][1]["properties"]["datasource"]["raw"]["iata"]
        cities_pd.loc[index, "Airport Address"] = response["features"][1]["properties"]["address_line2"]
        cities_pd.loc[index, "Distance"] = response["features"][1]["properties"]["distance"]
        cities_pd.loc[index, "Website"] = response["features"][1]["properties"]["datasource"]["raw"]["website"]
    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {cities_pd.loc[index, 'Airport Name']}")

    # Log the search results
    print(f"nearest airport from {city}: {cities_pd.loc[index, 'Airport Name']}")

# Display sample data
cities_pd

Starting airport details search
website not found for Bankstown Airport
nearest airport from Sydney: Bankstown Airport
nearest airport from Melbourne: Melbourne Airport
nearest airport from Perth: Jandakot Airport
website not found for Mareeba Airport
nearest airport from Cairns: Mareeba Airport
iata not found for Parafield Airport
nearest airport from Adelaide: Parafield Airport
iata not found for Heck Field Airport
nearest airport from Gold Coast: Heck Field Airport
nearest airport from Townsville: RAAF Base Townsville
iata not found for Wallsair Airstrip
nearest airport from Williamtown: Wallsair Airstrip


,City,Lat,Lon,Airport Name,IATA Name,Airport Address,Distance,Website
0,Sydney,-33.869844,151.208285,Bankstown Airport,BWU,"Airport Avenue, Bankstown Aerodrome NSW 2200, ...",19799,https://www.sydneyairport.com.au/
1,Melbourne,-37.814245,144.963173,Melbourne Airport,MEL,"Melbourne Airport VIC 3045, Australia",16122,https://www.melbourneairport.com.au/
2,Perth,-31.955893,115.860585,Jandakot Airport,JAD,"Compass Road, Jandakot WA 6149, Australia",14151,https://www.jandakotairport.com.au/
3,Cairns,-16.920666,145.772185,Mareeba Airport,MRG,"Beaufort Road, Mareeba Shire QLD, Australia",39698,
4,Adelaide,-34.928181,138.599931,Parafield Airport,ADL,"Adelaide Airport SA 5950, Australia",4898,https://www.adelaideairport.com.au/
5,Gold Coast,-28.002373,153.414599,Heck Field Airport,OOL,"Loongana Avenue, Bilinga QLD 4225, Australia",18807,https://www.goldcoastairport.com.au/
6,Townsville,-19.256939,146.823954,RAAF Base Townsville,TSV,"Freshwater Lagoon Road, Townsville QLD 4810, A...",5331,https://www.townsvilleairport.com.au/
7,Williamtown,-32.815,151.842778,Wallsair Airstrip,NTL,"55 Slades Road, Williamtown NSW 2318, Australia",620,https://www.newcastleairport.com.au


In [ ]:
# Save Data to CSV
